# 로지스틱 회귀

선형회귀의 예측값은 수치값으로 나와서 참 또는 거짓을 분류하는 문제에 적합하지 않다.  
로지스틱 회귀 모델은 선형 회귀의 결과를 입력값으로 받아 특정 레이블로 분류하는 모델이다.

### 로지스틱 회귀 학습
로지스틱 회귀 역시 경사하강법으로 최적의 w를 찾아내지만 비용함수는 평균제곱오차가 아닌 크로스 엔트로피(cross entropy)를 사용한다.  
그 이유는 바로 비선형성을 지니고 있는 시그모이드 함수 때문이다.

### 선형 vs 비선형
함수가 선형일 경우 평균제곱오차 비용함수는 아래쪽으로 볼록한 함수이다. 따라서 경사하강법을 사용해 최저의 에러를 갖는 회귀계수를 찾을 수 있다.  
하지만 로지스틱 회귀의 평균제곱오차는 아래쪽으로 볼록한 함수가 아니다. 그 이유는 로지스틱 회귀는 선형회귀 + 시그모이드 함수의 형태를 띠고 있으며,  
시그모이드 함수의 영향으로 평균제곱오차는 한 개 이상의 로컬 미니멈을 가질 수 있기 때문이다.  
경사하강법으로 미분값이 0인 지점을 찾을 경우 운이 좋아서 글로벌 미니멈의 w를 찾을 수도 있지만 운이 나쁠 경우  
로컬 미니멈의 w로 모델 학습이 마무리 될 수 있다.  
모델을 학습할 때 최적의 w를 항상 찾는다는 보장이 없으므로 평균제곱오차는 로지스틱 회귀의 적합한 비용함수가 아니다.

### 크로스 엔트로피
로지스틱 회귀 분류 모델은 주로 크로스 엔트로피를 비용함수로 사용한다.  
크로스 엔트로피란 서로 다른 두 확률 분포의 차이를 의미한다. 
로지스틱 회귀 관점에서는 모델의 예측값의 확률과 실제값 확률의 차이이다.  
예측값과 실제값의 차이를 가장 작게 하는 w를 구함으로써 최적의 w를 구하는 방법이 바로 크로스 엔트로피이다.

### 소프트맥스
다중 분류 로지스틱 회귀는 보통 소프트맥스라고 한다. 보통 로지스틱 회귀가 '예', '아니오'처럼 이진 분류하는 데 사용되지만  
로지스틱 회귀를 여러 개 붙여서 정규화된 출력을 함으로써 여러 개의 값을 분류할 수 있다.

# 단일 입력 로지스틱 회귀 모델
1개의 입력을 받아 0 또는 1을 출력하는 로지스틱 회귀 모델을 케라스로 구현해본다.

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation
import numpy as np

### 로지스틱 회귀 모델 만들기
sigmoid(wx+b)의 형태를 갖는 간단한 로지스틱 회귀를 케라스로 구현한다.

In [2]:
model = Sequential()
# 입력 1개를 받아 출력 1개를 리턴하는 선형 회귀 레이어를 생성한다.
model.add(Dense(input_dim=1, units=1))
# 선형 회귀의 출력값을 시그모이드에 연결한다
model.add(Activation('sigmoid'))
# 크로스 엔트로피를 비용함수로 설정해 경사하강법으로 학습한다.
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['binary_accuracy'])

### 데이터 생성
모델 학습을 위한 데이터를 생성한다.

In [3]:
X = np.array([-2, -1.5, -1, 1.25, 1.62, 2])
Y = np.array([0, 0, 0, 1, 1, 1])

### 모델 학습
300번의 반복 학습을 통해 최적의 w와 b를 찾는다.

In [4]:
model.fit(X, Y, epochs=300, verbose=0)

학습 데이터에 따른 실제 모델의 출력값을 확인해본다.  
-2, -1.5, -1에 해당하는 값은 0.5보다 작고, 나머지 값들은 0.5보다 큰 것을 확인할 수 있다.

In [5]:
model.predict([-2, -1.5, -1, 1.25, 1.62, 2])

array([[0.05117321],
       [0.10003212],
       [0.18637702],
       [0.8557482 ],
       [0.9101559 ],
       [0.94609547]], dtype=float32)

시그모이드 특성상 왼쪽 극한의 값은 0으로 수렴하며, 오른쪽 극한의 값은 1로 수렴하는 것을 확인할 수 있다.

In [6]:
model.predict([-1000, 1000])

array([[0.],
       [1.]], dtype=float32)

### 모델 요약
model.summary() 명령어를 사용하여, 모델이 어떻게 구성되어 있는지 확인할 수 있다.  
단일 입력 로지스틱 모델은 단 한개의 w와 b가 첫 번째 레이어에 존재하며, 학습 과정을 통해 최적의 w와 bias가 지정된다.  
dense 레이어에 2개의 param은 w와 b이며, dense 레이어가 바로 선형 회귀 레이어이다.  
선형 회귀 레이어의 출력값은 activation 레이어의 입력으로 들어가며, activation 레이어는 시그모이드 함수로 설정한다.

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 2         
_________________________________________________________________
activation (Activation)      (None, 1)                 0         
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


첫 번째 레이어에 존재하는 w와 b는 다음 명령어로 확인할 수 있다.

In [8]:
model.layers[0].weights

[<tf.Variable 'dense/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[1.4462848]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(1,) dtype=float32, numpy=array([-0.0274407], dtype=float32)>]

학습을 통해 구한 최적의 w와 b는 get_weights() 함수로 확인할 수 있다.

In [9]:
model.layers[0].get_weights()

[array([[1.4462848]], dtype=float32), array([-0.0274407], dtype=float32)]